Steps to Perform for MFCC Transfer Learning
Step 1: Load the files and convert to mfcc

In [60]:
# import numpy as np
# import librosa as lp

# y, sr = lp.load('audio-train-new/a_country_doctor_1504_librivox_64kb_mp3/split/12/wav/000.wav')
# y = y[::3]
# print y.shape
# mfcc = lp.feature.mfcc(y=y, sr=sr, n_mfcc=15)
# print mfcc.shape

In [61]:
# import matplotlib.pyplot as plt
# import librosa.display

# plt.figure(figsize=(10, 4))
# lp.display.specshow(mfcc, x_axis='time')
# plt.tight_layout()

In [62]:
# import matplotlib.pyplot as plt
# import librosa.display

# plt.figure(figsize=(10, 4))
# lp.display.specshow(mfcc, x_axis='time')
# plt.tight_layout()

In [63]:
import matplotlib.pyplot as plt
import os
import subprocess

def run_preprocess(root, length, split):
    for subdir, dirs, files in os.walk(root):
        for directory in dirs:
            subprocess.call(["./preprocess", os.path.join(subdir, directory), length, split])
        break

In [64]:
import numpy as np
import librosa as lp

def load_features(root, split):
    mfcc_data = []
    mfcc_label = []
    for subdir, dirs, files in os.walk(root):
        for directory in dirs:
            file_path = os.path.join(subdir, directory, "split", split, "wav")
            print directory
            for filename in os.listdir(file_path):
                y, sr = lp.load(os.path.join(file_path, filename))
                mfcc = lp.feature.mfcc(y = y, sr = 16000, n_mfcc = 15)
#                 print mfcc.shape
                mfcc = np.pad(mfcc, pad_width=((0, 0), (0, 122)), mode='constant')
                if mfcc.shape != (15, 122):
                    mfcc = mfcc[:, :122]
                mfcc_data.append(mfcc)
                mfcc_label.append(directory)
        break
    return mfcc_data, mfcc_label

In [65]:
import shutil

def cleanup_split(root):
    for subdir, dirs, files in os.walk(root):
        for directory in dirs:
            shutil.rmtree(os.path.join(subdir, directory, "split"), ignore_errors = True)
        break

In [66]:
cleanup_split('audio-train-new')
run_preprocess('audio-train-new', "120", "3")

In [67]:
X, y = load_features('audio-train-new', "5")

a_country_doctor_1504_librivox_64kb_mp3
a_dreamers_tales_dm_1501_librivox_64kb_mp3
adventuresbobwhite_1505_librivox_64kb_mp3
aequanimitas_1412_librivox_64kb_mp3
agnesgreyversion3_1501_librivox_64kb_mp3
airplaneflyinghandbookvol3_1409_librivox_64kb_mp3
alondonlife_1412_librivox_64kb_mp3
ancient_modern_celebrated_freethinkers_1503_librivox_64kb_mp3
aprendizdeconspirador_1412_librivox_64kb_mp3
aristotles_masterpiece_1506_librivox_64kb_mp3
artofdivinecontentment_1512_librivox_64kb_mp3
battle-pieces_aspects_war_1501_librivox_64kb_mp3
blackriders_1410_librivox_64kb_mp3
boatsoftheglencarrig_1411_librivox_64kb_mp3
bookofgoodcounsels_1506_librivox_64kb_mp3
briefe_paulus_auswahl_1506_librivox_64kb_mp3
britishsubject_president_1505_librivox_64kb_mp3
canti_leopardi_1512_librivox_64kb_mp3
childs_garden_of_verses_1503_librivox_64kb_mp3
company_b_307th_infantry_wc_1412_librivox_64kb_mp3
contes_humoristiques_1505_librivox_64kb_mp3
curlyandfloppytwistytail_1509_librivox_64kb_mp3
de_reis_naar_de_maan_15

In [1]:
import pickle

# with open('mfcc_X_y_45_3', 'rb') as f:
#     X, y = pickle.load(f)

# with open('mfcc_X_y_120_5', 'wb') as f:
#     pickle.dump([X, y], f)

(15, 122)


In [69]:
X = np.dstack(X)

X = np.swapaxes(X, 0, 1)
X = np.swapaxes(X, 0, 2)

X = X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)

In [78]:

print len(set(y))

118


In [70]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=64)

In [71]:
import keras
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

l_enc = LabelEncoder()
l_enc.fit(y_train)
y_train_enc = l_enc.transform(y_train)
y_train_norm = np_utils.to_categorical(y_train_enc)

l_enc.fit(y_test)
y_test_enc = l_enc.transform(y_test)
y_test_norm = np_utils.to_categorical(y_test_enc)

In [72]:
# del X, y, y_train_enc, y_test_enc, y_train, y_test

In [79]:
from keras.layers import Activation, Conv2D, Dense, Dropout, Flatten, MaxPooling2D
from keras.models import Sequential

model = Sequential()
model.add(Conv2D(8, (3, 3), padding='same',
                 input_shape=(15, 221, 1)))
model.add(Activation('relu'))
model.add(Conv2D(8, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(16, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(16, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Activation('relu'))
model.add(Dense(118))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [81]:
# epoch = 1
# while epoch <= 30:
model.fit(np.array(X_train), y_train_norm,
          batch_size=32,
          epochs=20,
          verbose=1,
          shuffle = True,
         validation_data=(np.array(X_test), y_test_norm))
#     model.save_weights('spect_model_weights_' + str(epoch) + '.h5')
#     epoch += 1

Train on 1743 samples, validate on 748 samples
Epoch 1/20
1743/1743 [==============================] - 6s 3ms/step - loss: 0.5583 - acc: 0.8371 - val_loss: 3.6973 - val_acc: 0.2179
Epoch 2/20
1743/1743 [==============================] - 6s 3ms/step - loss: 0.4964 - acc: 0.8480 - val_loss: 3.6273 - val_acc: 0.2406
Epoch 3/20
1743/1743 [==============================] - 6s 3ms/step - loss: 0.4616 - acc: 0.8612 - val_loss: 3.7200 - val_acc: 0.2420
Epoch 4/20
1743/1743 [==============================] - 6s 3ms/step - loss: 0.4232 - acc: 0.8807 - val_loss: 3.7781 - val_acc: 0.2353
Epoch 5/20
1743/1743 [==============================] - 6s 3ms/step - loss: 0.3616 - acc: 0.8910 - val_loss: 3.7911 - val_acc: 0.2540
Epoch 6/20
1743/1743 [==============================] - 6s 4ms/step - loss: 0.3046 - acc: 0.9053 - val_loss: 3.7813 - val_acc: 0.2460
Epoch 7/20
1743/1743 [==============================] - 6s 3ms/step - loss: 0.2737 - acc: 0.9220 - val_loss: 3.8139 - val_acc: 0.2607
Epoch 8/20
1743

In [82]:
model.save_weights('mfcc_model_weights_120_5.h5')